In [1]:
!pip install -U efficientnet

Looking in indexes: https://artifacts.mitre.org/artifactory/api/pypi/python/simple
^C
ERROR: Operation cancelled by user


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# visulization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import os
import gc # garbage collection
import glob # extract path via pattern matching
import random
import math
import cv2 # read image
# store to disk
import pickle
import h5py # like numpy array


from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

from keras.models import Sequential, Model
from keras.models import load_model
from keras.layers import Input, Dense, Conv2D, MaxPool2D, AveragePooling2D
from keras.layers import Flatten, Dropout, BatchNormalization, Activation
from keras.layers import Add
from keras.optimizers import SGD, RMSprop, Adam
from keras import regularizers
import keras

from keras.callbacks import ModelCheckpoint, EarlyStopping

In [4]:
ROOT_DIR = './state-farm-distracted-driver-detection/'
TRAIN_DIR = ROOT_DIR + 'imgs/train/'
TEST_DIR = ROOT_DIR + 'imgs/test/'
driver_imgs_list = pd.read_csv(ROOT_DIR + "driver_imgs_list.csv")
sample_submission = pd.read_csv(ROOT_DIR + "sample_submission.csv")

In [5]:
def get_image(path, img_height=None, img_width=None, rotate=False, color_type=0):
    img = cv2.imread(path, color_type)
    if img_width and img_height:
        img = cv2.resize(img, (img_width, img_height))
    if rotate is True:
        rows, cols = img.shape
        rotation_angle = random.uniform(10,-10)
        M = cv2.getRotationMatrix2D((cols/2, rows/2), rotation_angle, 1)
        img = cv2.warpAffine(img, M, (cols,rows))
    return img

In [6]:
random_list = np.random.permutation(len(driver_imgs_list))[:50]
df_copy = driver_imgs_list.iloc[random_list]
image_paths = [TRAIN_DIR+row.classname+'/'+row.img 
                   for (index, row) in df_copy.iterrows()]
image_shapes = [get_image(path).shape for path in image_paths]
print(set(image_shapes))

{(480, 640)}


In [7]:
img_path_list = []
label_list = []
for index, row in driver_imgs_list.iterrows():
    img_path_list.append('{0}{1}/{2}'.format(TRAIN_DIR, row.classname, row.img))
    label_list.append(int(row.classname[1]))
# One hot vector representation of labels
y_labels_one_hot = to_categorical(label_list, dtype=np.int8)
x_img_path = np.array(img_path_list)

In [8]:
from sklearn.utils import shuffle

np.save('x_img_path.npy', x_img_path)
np.save('y_labels_one_hot.npy', y_labels_one_hot)

x_img_path_shuffled, y_labels_one_hot_shuffled = shuffle(x_img_path, y_labels_one_hot)

# saving the shuffled file.
# you can load them later using np.load().
np.save('y_labels_one_hot_shuffled.npy', y_labels_one_hot_shuffled)
np.save('x_img_path_shuffled.npy', x_img_path_shuffled)

In [9]:
from sklearn.model_selection import train_test_split
# Used this line as our filename array is not a numpy array.
x_img_path_shuffled_numpy = np.array(x_img_path_shuffled)

X_train_filenames, X_val_filenames, y_train, y_val = train_test_split(
    x_img_path_shuffled_numpy, y_labels_one_hot_shuffled, test_size=0.2, random_state=1)

print(X_train_filenames.shape) # (3800,)
print(y_train.shape)           # (3800, 12)

print(X_val_filenames.shape)   # (950,)
print(y_val.shape)             # (950, 12)

# You can save these files as well. As you will be using them later for training and validation of your model.
np.save('X_train_filenames.npy', X_train_filenames)
np.save('y_train.npy', y_train)

np.save('X_val_filenames.npy', X_val_filenames)
np.save('y_val.npy', y_val)

(17939,)
(17939, 10)
(4485,)
(4485, 10)


In [10]:
IMG_HEIGHT = 64
IMG_WIDTH = 64
BATCH_SIZE = 32
CHANNEL = 3
class Img_Generator(keras.utils.Sequence):
    def __init__(self, image_filenames, labels, batch_size) :
        self.image_filenames = image_filenames
        self.labels = labels
        self.batch_size = batch_size
    def __len__(self) :
        return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)
    def __getitem__(self, idx) :
        batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
        img_list = []
        for file_name in batch_x:
            if CHANNEL == 1:
                original_img = cv2.imread(file_name, 0)
            else:
                original_img = cv2.imread(file_name, 1)
            im = cv2.resize(original_img, (IMG_HEIGHT, IMG_WIDTH))
            #color = [0, 0, 0]
            #new_im = cv2.copyMakeBorder(im, 40, 40, 0, 0, cv2.BORDER_CONSTANT, value=color)
            #im = cv2.resize(new_im, (224, 224))
            img_list.append(im)
        img_batch = np.array(img_list)
        if CHANNEL == 1:
            img_batch = np.expand_dims(img_batch, axis=-1)
        return img_batch, np.array(batch_y)

In [11]:
# size of input [BATCH_SIZE, IMG_HEIGHT, IMG_WIDTH, CHANNEL]
train_gen = Img_Generator(X_train_filenames, y_train, BATCH_SIZE)
val_gen = Img_Generator(X_val_filenames, y_val, BATCH_SIZE)

In [12]:
!rm -f saved_models/weights_best_efficient.hdf5

In [13]:
from keras import optimizers
from efficientnet.keras import EfficientNetB5
from keras.layers import (Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D,
                          BatchNormalization, Input, Conv2D, GlobalAveragePooling2D,concatenate,Concatenate,multiply, LocallyConnected2D, Lambda)

nb_train_samples = 17943
nb_validation_samples = 4481

IMAGENET_WEIGHTS_HASHES = {
    'efficientnet-b5': ('30172f1d45f9b8a41352d4219bf930ee'
                        '3339025fd26ab314a817ba8918fefc7d',
                        '9d197bc2bfe29165c10a2af8c2ebc675'
                        '07f5d70456f09e584c71b822941b1952')
}
IMAGENET_WEIGHTS_PATH = (
    'https://github.com/Callidior/keras-applications/'
    'releases/download/efficientnet/')

eff_net = EfficientNetB5(weights=None,
                        include_top=False,
                        input_shape=(IMG_WIDTH, IMG_HEIGHT, CHANNEL))

in_lay = Input(shape=(64,64,3))

file_name = "efficientnet-b5" + '_weights_tf_dim_ordering_tf_kernels_autoaugment_notop.h5'
file_hash = IMAGENET_WEIGHTS_HASHES["efficientnet-b5"][1]
weights_path = keras.utils.get_file(
            file_name,
            IMAGENET_WEIGHTS_PATH + file_name,
            cache_subdir='models',
            file_hash=file_hash,
 )

eff_net.load_weights(weights_path)
pt_depth = eff_net.get_output_shape_at(0)[-1]
pt_features = eff_net(in_lay)
bn_features = BatchNormalization()(pt_features)

In [14]:
from keras.layers import (Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D,
                          BatchNormalization, Input, Conv2D, GlobalAveragePooling2D,concatenate,Concatenate,multiply, LocallyConnected2D, Lambda)
# Attention mechanism to turn pixels in the GAP on and off
attn_layer = Conv2D(64, kernel_size = (1,1), padding = 'same', activation = 'relu')(Dropout(0.5)(bn_features))
attn_layer = Conv2D(16, kernel_size = (1,1), padding = 'same', activation = 'relu')(attn_layer)
attn_layer = Conv2D(8, kernel_size = (1,1), padding = 'same', activation = 'relu')(attn_layer)
attn_layer = Conv2D(1, kernel_size = (1,1), padding = 'valid', activation = 'sigmoid')(attn_layer)

mask_features = multiply([attn_layer, bn_features])
gap_features = GlobalAveragePooling2D()(mask_features)
gap_mask = GlobalAveragePooling2D()(attn_layer)
# to account for missing values from the attention model
gap = Lambda(lambda x: x[0]/x[1], name = 'RescaleGAP')([gap_features, gap_mask])
gap_dr = Dropout(0.25)(gap)
dr_steps = Dropout(0.25)(Dense(128, activation = 'relu')(gap_dr))
out_layer = Dense(10, activation = 'softmax')(dr_steps)
retina_model = Model(inputs = [in_lay], outputs = [out_layer])

In [15]:
retina_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
efficientnet-b5 (Model)         (None, 2, 2, 2048)   28513520    input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 2, 2, 2048)   8192        efficientnet-b5[1][0]            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 2, 2, 2048)   0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [16]:
from keras.callbacks import (ModelCheckpoint, LearningRateScheduler,
                             EarlyStopping, ReduceLROnPlateau,CSVLogger)

epochs = 15; batch_size = 32
checkpoint = ModelCheckpoint('./model_.h5', monitor='loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)
reduceLROnPlat = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=4, 
                                   verbose=1, mode='auto', epsilon=0.0001)
early = EarlyStopping(monitor="loss", 
                      mode="min", 
                      patience=9)
csv_logger = CSVLogger(filename='./training_log.csv',
                       separator=',',
                       append=True)

train_generator = train_gen
# train_mixup = My_Generator(train_x, train_y, batch_size, is_train=True, mix=False, augment=True)
valid_generator = val_gen

/opt/anaconda3/envs/snowflakes/lib/python3.7/site-packages/keras/callbacks/callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [17]:
from keras.callbacks import Callback
class QWKEvaluation(Callback):
    def __init__(self, validation_data=(), batch_size=64, interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.batch_size = batch_size
        self.valid_generator, self.y_val = validation_data
        self.history = []

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict_generator(generator=self.valid_generator,
                                                  steps=np.ceil(float(len(self.y_val)) / float(self.batch_size)),
                                                  workers=1, use_multiprocessing=False,
                                                  verbose=1)
            def flatten(y):
                return np.argmax(y, axis=1).reshape(-1)
            
            score = cohen_kappa_score(flatten(self.y_val),
                                      flatten(y_pred),
                                      labels=[0,1,2,3,4],
                                      weights='quadratic')
            print("\n epoch: %d - QWK_score: %.6f \n" % (epoch+1, score))
            self.history.append(score)
            if score >= max(self.history):
                print('saving checkpoint: ', score)
                self.model.save('model_bestqwk.h5')

qwk = QWKEvaluation(validation_data=(valid_generator, y_val),
                    batch_size=batch_size, interval=1)

In [18]:
for layer in retina_model.layers:
    layer.trainable = False

for i in range(-3,0):
    retina_model.layers[i].trainable = True

retina_model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(1e-3),metrics=['accuracy'])

retina_model.fit_generator(
    train_generator,
    steps_per_epoch=np.ceil(float(len(y_train)) / float(128)),
    epochs=2,
    workers=2, use_multiprocessing=True,
    verbose=1,
    callbacks=[early, checkpoint])

Epoch 1/2
141/141 [==============================] - 55s 388ms/step - loss: 2.5187 - accuracy: 0.1849

Epoch 00001: loss improved from inf to 2.51903, saving model to ./model_.h5
Epoch 2/2
141/141 [==============================] - 47s 330ms/step - loss: 2.2053 - accuracy: 0.2580

Epoch 00002: loss improved from 2.51903 to 2.20531, saving model to ./model_.h5


In [19]:
def kappa_loss(y_true, y_pred, y_pow=2, eps=1e-12, N=10, bsize=32, name='kappa'):
    with tf.name_scope(name):
        y_true = tf.to_float(y_true)
        repeat_op = tf.to_float(tf.tile(tf.reshape(tf.range(0, N), [N, 1]), [1, N]))
        repeat_op_sq = tf.square((repeat_op - tf.transpose(repeat_op)))
        weights = repeat_op_sq / tf.to_float((N - 1) ** 2)
    
        pred_ = y_pred ** y_pow
        try:
            pred_norm = pred_ / (eps + tf.reshape(tf.reduce_sum(pred_, 1), [-1, 1]))
        except Exception:
            pred_norm = pred_ / (eps + tf.reshape(tf.reduce_sum(pred_, 1), [bsize, 1]))
    
        hist_rater_a = tf.reduce_sum(pred_norm, 0)
        hist_rater_b = tf.reduce_sum(y_true, 0)
    
        conf_mat = tf.matmul(tf.transpose(pred_norm), y_true)
    
        nom = tf.reduce_sum(weights * conf_mat)
        denom = tf.reduce_sum(weights * tf.matmul(
            tf.reshape(hist_rater_a, [N, 1]), tf.reshape(hist_rater_b, [1, N])) /
                              tf.to_float(bsize))
    
        return nom*0.5 / (denom + eps) + categorical_crossentropy(y_true, y_pred)*0.5

In [27]:
import tensorflow as tf
from keras.losses import binary_crossentropy, categorical_crossentropy
from sklearn.metrics import f1_score, fbeta_score, cohen_kappa_score

for layer in retina_model.layers:
    layer.trainable = True
callbacks_list = [checkpoint, csv_logger, reduceLROnPlat, early, qwk]
retina_model.compile(
            loss='categorical_crossentropy',
            optimizer=Adam(lr=1e-4))
retina_model.fit_generator(
    train_generator,
    steps_per_epoch=np.ceil(float(len(y_train)) / float(batch_size)),
    validation_data=valid_generator,
    validation_steps=np.ceil(float(len(y_val)) / float(batch_size)),
    epochs=epochs,
    verbose=1,
    workers=1, use_multiprocessing=False,
    callbacks=callbacks_list)

Epoch 1/15
561/561 [==============================] - 1346s 2s/step - loss: 0.3980 - val_loss: 0.0023

Epoch 00001: val_loss improved from 0.03256 to 0.00233, saving model to ./model_.h5
141/141 [==============================] - 59s 418ms/step

 epoch: 1 - QWK_score: 0.982859 

saving checkpoint:  0.982858592150755
Epoch 2/15
561/561 [==============================] - 1270s 2s/step - loss: 0.1652 - val_loss: 1.3542e-05

Epoch 00002: val_loss improved from 0.00233 to 0.00001, saving model to ./model_.h5
141/141 [==============================] - 55s 391ms/step

 epoch: 2 - QWK_score: 0.989366 

saving checkpoint:  0.9893659354854508
Epoch 3/15
561/561 [==============================] - 1244s 2s/step - loss: 0.0876 - val_loss: 5.3644e-06

Epoch 00003: val_loss improved from 0.00001 to 0.00001, saving model to ./model_.h5
141/141 [==============================] - 1303s 9s/step

 epoch: 3 - QWK_score: 0.993664 

saving checkpoint:  0.9936644811410007
Epoch 4/15
561/561 [=================